# Set up

In [29]:
import requests
import json, os

nn_id = 'nn982798'  # put some key value you want to test

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")
print("done")

done


# Net & Version Create

In [30]:
####(1) 네트워크 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/',
                     json={
                         "biz_cate": "MES",
                         "biz_sub_cate": "M60",
                         "nn_title" : "test",
                         "nn_desc": "test desc",
                         "use_flag" : "Y",
                         "dir": "purpose?",
                         "config": "N"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

####(2) 버전 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/version/',
                 json={
                     "nn_def_list_info_nn_id": "",
                     "nn_wf_ver_info": "test version info",
                     "condition": "1",
                     "active_flag": "Y"
                 })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : nn982798
evaluation result : nn982798


# Graph Flow 정의
CSV 데이터를 AutoEncoder 에 훈련시키는 Graph Flow 정의, Eval Node 는 정의하지 않음. 

In [31]:
# Work Flow 틀을 구성하도로고 지시한다. (정해진 틀을 강제로 생성)
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/' + nn_id +'/wfver/1/',
                     json={
                         "type": 'autoencoder_csv'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : autoencoder_csv


# CSV Data Upload 
한글 CSV 데이터를 업로드 한다. 

In [32]:
return_dict = {}
return_dict['test'] = open('../../data/seq2seq_mansearch_3.csv', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/datasrc/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']


# Data Node 정의
데이터 처리에 관한 정의 CSV 파일을 Pandas 를 통해서 읽고 HDF5 로 변환, 별도의 전처리는 지정하지 않음 

In [33]:
# (1) Train Data Node 의 속성을 정의
# 어디서 Source 를 어떻게 가지고 올것인지 파라메터 정의 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/datasrc/',
                     json={
                         "type": "csv",
                         "source_server": "local",
                         "source_sql": "all",
                     })

# 전처리는 어떤 것을 할지 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/datasrc/',
                      json={
                          "preprocess":  "none",
                      })
# 전처리가 완료된 데이터는 어디에 저장을 할지 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/datasrc/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : /hoya_str_root/nn982798/1/datasrc


# Data Feeder 
Network 에서 훈련을 위해 원하는 형태로 데이터를 편집하여 제공하는 기능 수행 

In [34]:
# (2) Network 에 데이터를 Feed하는 Node 의 속성을 정의 
resp = requests.post('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/frame/net/autoencoder/nnid/'+nn_id+'/ver/1/node/feed_train/',
                     json={
                         "encode_column" : "encode",
                         "encode_len" : 10,
                         "vocab_size" : 100,
                         "preprocess": "mecab",
                         "embed_type" : 'onehot'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'encode_column': 'encode', 'encode_len': 10, 'preprocess': 'mecab', 'embed_type': 'onehot', 'vocab_size': 100}


# AutoEncoder 정의 
n_hidden 의 경우 Encoder 부만 정의하면 Decoder부는 동일한 Vector 를 Revsere로 활용하도록 되어 있다. 

In [35]:
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/autoencoder/nnid/' + nn_id + '/ver/1/node/netconf_node/',
                     json={
                        "learning_rate" : 0.01,
                        "iter" : 10,
                        "batch_size" : 10,
                        "examples_to_show" : 10,
                        "n_hidden" : [200, 100] 
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'learning_rate': 0.01, 'model_path': '/hoya_model_root/nn982798/1/netconf_node', 'n_hidden': [200, 100], 'batch_size': 10, 'iter': 10, 'examples_to_show': 10}


# Train 을 시작한다. 

In [36]:
# Run All Workflow
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/1/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : [None, None, 'nn982798_1_netconf_node']


# Compressed Vector 를 구해보자 
해당 Vector 의 Size 는 "n_hidden" : [200, 100, 10]에서 정의한 마지막 Layer 의 사이즈가 될 것이다. 

In [38]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/autoencoder/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "드래곤은 무엇인가?" ,
                           "type" : "encoder"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data[0][0:10]))

evaluation result : [0.6689520478248596, 1.7301836123806424e-05, 0.9937833547592163, 1.895393870654516e-05, 0.9999933242797852, 1.3650051187141798e-05, 0.31484848260879517, 0.9999873638153076, 0.75555419921875, 0.9984480142593384]


# 복원된 Vector 추출 
복원된 Vector는 사전 Filter 개념으로 이해하고 다른 네트워크의 인풋으로도 사용할 수 있다. 

In [61]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/autoencoder/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름]의 [직책] 조회" ,
                           "type" : "decoder"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data[0][0:10]))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/autoencoder/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "전혀 알 수 없는 내용" ,
                           "type" : "decoder"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data[0][0:10]))

evaluation result : [3.931156129510782e-07, 0.9999997615814209, 4.723608526546741e-06, 0.0035316667053848505, 4.7008197157083487e-07, 5.42258785571903e-05, 0.0019018376478925347, 6.032691146629077e-08, 5.0723610911518335e-06, 3.4362684139116473e-12]
evaluation result : [6.858207370896707e-07, 0.9999998807907104, 2.62475514318794e-05, 0.00033533188980072737, 7.07373772002029e-07, 0.0003725199494510889, 3.448012284934521e-05, 2.458450865105988e-07, 5.984585413898458e-07, 7.403353646275035e-13]


# 복원된 Vector 와  입력된 Vector 유사성 테스트 
입력된 Vector 와 복원된 Vector 의 유사성을 근거로 Anomaly Detection 을 수행한다 .<br>
위의 두개가 훈련에 사용한 데이터이고 나머지는 훈련에 사용하지 않은 데이터이다. 지기 복원율을 구하는 방식으로 테스트한 결과 현재로써는 챗봇에 어느정도 사용할 수 있을 것이라고 생각된다 .

In [103]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/anomaly/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름]의 [직책] 조회" }
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data)) 

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/anomaly/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[부서] [이름] [직급]"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/anomaly/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름]의 [직책] 찾아줘"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/anomaly/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "가나다라마바사"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/anomaly/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "굉장히 이상한 내용 !!!!"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/anomaly/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "히키다타하아하아다가"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/anomaly/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "하하하 너는 바보니?"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : 0.4383151852794167
evaluation result : 0.5246161067748699
evaluation result : 0.3529608467122519
evaluation result : 0.3090275161664491
evaluation result : 0.19375176775056424
evaluation result : 0.17096916001501405
evaluation result : 0.27226062018619923
